In [1]:
from fastai import *
from fastai.text import *

In [ ]:
! kaggle competitions download -c quora-insincere-questions-classification -f train.csv
! kaggle competitions download -c quora-insincere-questions-classification -f test.csv
! mkdir data/quora
#! unzip train.csv.zip -d data/quora
#! unzip test.csv.zip -d data/quora
! chmod a+r data/quora/*

In [2]:
path = './data/'

In [35]:
df = pd.read_csv(path+'train.csv')
df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [21]:
sample = df.sample(3000)
sample.to_csv(path+'quora_sample.csv')

In [22]:
lm_data = TextLMDataBunch.from_csv(Path(path), 'quora_sample.csv', text_cols='question_text')

In [23]:
lm_data.show_batch()

idx,text
0,xxfld 1 why do people of xxunk xxunk other xxunk and hindi ? xxfld 1 how bitcoin is a xxunk that will explode ? xxfld 1 is being ' strong - xxunk ' a quality ? xxfld 1 the xxunk xxunk has been xxunk now . how do you feel ? xxfld 1 what do you think is the most xxunk xxunk ? xxfld 1 how 's xxunk capital ? xxfld 1 why do spanish xxunk look like xxunk xxunk ? xxfld 1 what was the difference between computer from 10 years ago than today
1,xxunk xxunk xxunk to talk about things that she does n’t know a xxunk thing about . when will people stop xxunk to her ? xxfld 1 how can you create xxunk income sources while working on xxunk in the us ? xxfld 1 are there xxunk people in korea ? xxfld 1 why do religious jews xxunk for the xxunk of the third temple ? what was so xxunk about the era of the first two ? xxfld 1 why some felt love with someone xxunk they are married ? xxfld 1 why is
2,"a facebook account and it got xxunk . what should i do to get back on ? xxfld 1 what is the technology used to develop xxup usb type - c ? xxfld 1 did christian xxunk ( xxunk xxunk ) xxunk right after xxunk xxunk in a road xxunk ? does that mean that the following was only his xxunk ? xxfld 1 why should the amount of salt in foods decrease when you reach your xxunk ? xxfld 1 who xxunk you the most , and why ? xxfld 1 how xxunk can"
3,"xxunk of army children in xxup xxunk ? xxfld 1 what is the future of software engineering in pakistan ? xxfld 1 what is the new campus xxunk procedure of xxup tcs ? xxfld 1 how many countries have tax xxunk number ? xxfld 1 why did xxunk xxunk ? xxfld 1 is xxunk a term of the xxunk xxunk , 4 , xxunk ? xxfld 1 what are you going to use your google home xxunk for ? xxfld 1 i like xxunk , xxunk - xxunk , and xxunk . what tv shows"
4,xxfld 1 how can we get xxunk to improve our future xxunk ? xxfld 1 what is xxup xxunk ? xxfld 1 what was your experience of spending xxunk years in xxunk ? xxfld 1 being a xxunk person i can not enjoy with my xxunk friends . but he wants both me and them . xxunk should i do ? xxfld 1 when does the truth start to become a xxunk ? xxfld 1 do muslim men xxunk watch porn ? xxfld 1 what is xxunk xxunk known for ? xxfld 1 how can
5,1 how can one xxunk themselves against xxunk ( of any kind ) ? xxfld 1 are there any non hip - xxunk people who xxunk xxunk ? xxfld 1 are there any website or app from where we can get xxunk for financial services xxunk company ? or any other xxunk for getting business ? xxfld 1 are xxunk university certifications valid ? xxfld 1 what are the extra benefits when an xxunk institute have a website ? xxfld 1 what is the best line to live the best life ? xxfld 1 can
6,"main xxunk of the xxup usb 3 port over a xxup usb 3 port ? xxfld 1 what is your favorite episode of gravity xxunk ? xxfld 1 how about instead of xxunk the xxup xxunk , the xxunk , the parents , or the lack of gun control in the florida xxunk , we blame the xxunk ? xxfld 1 what is my xxunk xxunk ? xxfld 1 why would someone sit up and sleep in the xxunk of the night ? xxfld 1 what skill have you “ lost ” over the years"
7,? xxfld 1 what is a good way to learn xxunk legal principles ? xxfld 1 how can i xxunk money from xxunk in xxunk ( xxunk ) ? xxfld 1 what are the best xxunk in xxunk for xxunk ? xxfld 1 where can i get french xxunk address ? xxfld 1 why are chinese using xxunk to eat ? do n't they know there are such things called xxunk and xxunk xxfld 1 what is xxunk xxunk of xxunk xxunk ? xxfld 1 what are the best practices of xxunk coding ? xxfld
8,"xxfld 1 have you ever set two of your friends up and they xxunk along xxunk and yet you xxunk felt xxunk because they xxunk on and away from you ? xxfld 1 which xxunk of the western history xxunk you most , and why ? xxfld 1 in case of a building with a xxunk xxunk what does street level means ? what is the difference between street level and ground level ? xxfld 1 if you do n't believe in life before life , why would you believe in life after death ?"
9,"t

In [16]:
lm_data.save('lmdata')

In [24]:
learn = language_model_learner(lm_data, drop_mult=0.3, pretrained_model=URLs.WT103)

In [25]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 

In [ ]:
learn.recorder.plot()

In [31]:
learn.fit_one_cycle(3, 1e-2, moms=(0.8, 0.7))

Total time: 01:00
epoch  train_loss  valid_loss  accuracy
1      3.506519    3.221165    0.373495  (00:21)
2      3.420686    3.169293    0.383464  (00:18)
3      3.355350    3.151510    0.380931  (00:19)



In [34]:
learn.predict(text="Asking this question because", n_words=20)

Total time: 00:01



'Asking this question because of the modern personal reasons for a strong . west of the team in less than many of digital history'

In [33]:
learn.save('lm_stage1')

In [36]:
data_clas = TextClasDataBunch.from_csv(path, 'train.csv', label_cols='target', text_cols='question_text')

In [38]:
data_clas.show_batch()

text,label
xxfld 1 what is [ math ] \ frac { \ xxunk } } \ tan ( \ tan ( { \ boxed { \ xxunk } \ sum _ { \ xxunk { \ infty } \ xxunk } \ sum _ { \ xxunk { \ infty },1
"xxfld 1 how do i ask my friend if he 's still interested in me ? he told me that he was going to ask me out weeks ago , however something personal happened and i 've been giving him space . i 'm also confused of how i should",0
"xxfld 1 i get these answer requests called "" best match from quora "" but i do n't see the "" pass "" option to make the notifications icon go away . how can i get rid of them aside from reporting those questions ? ( i remember there used",0
"xxfld 1 why do mothers act like they are special ? they just did what every other mother out there did , had sex , got pregnant , and had a kid . it 's literally what the human race does , it 's no miracle , so why act",1
"xxfld 1 what us universities can i apply with an ib score of 30 and xxup sat of 1400 ( reading 610 , math 790 ) ? i 'm thinking of skidmore , bates and colorado college , but do they accept an ib score of 30 ? i 'm",0


In [45]:
learn = text_classifier_learner(data_clas, drop_mult=0.5)

In [40]:
learn.lr_find()
learn.recorder.plot()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 

In [46]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy


KeyboardInterrupt: 